<a href="https://colab.research.google.com/github/dasmiq/cs6200-retrieval-models/blob/main/retrieval-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Models and Evaluation



In this notebook, you will evaluate results ranking on a test collection. First, you'll compute the mean average precision of a baseline BM25 model. Then you'll implement a query-likelihood retrieval model based on the same inverted index.

This notebook uses the [Pyserini](http://pyserini.io/) library, a Python interface to [Anserini](http://anserini.io) and thus to [Lucene](https://lucene.apache.org/), a widely-used open-source search engine. This library is written and maintained by Jimmy Lin and his colleagues at the University of Waterloo.


We start by installing the python interface. Since it calls the underlying Lucene search engine, we make sure we point to an appropriate Java installation. If you don't have Java 11, this would need to be changed.

In [1]:
!pip install pyserini
# You can change this to gpu if you have one.
!pip install faiss-cpu

# Point python at java if it can't find it.
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 104.3 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━

You can use the `SimpleSearcher` to search over an index. We can initialize the searcher with a pre-built index, which Pyserini will automatically download if it hasn't already:

In [2]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('robust04')

lucene-index.robust04.20221005.252b5e.tar.gz: 1.68GB [01:35, 18.9MB/s]                            


SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


Now we can search for a query and inspect the results:

In [3]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `hits` object also contains the raw text of the documents in the index before processing. In other words, this version of the text has not been divided into fields, tokens, etc.

In [4]:
hits[0].raw

'<DATE>\n<P>\nSeptember 27, 1990, Thursday, Ventura County Edition\n</P>\n</DATE>\n<HEADLINE>\n<P>\nHUNGRY WILDLIFE STRAYING INTO SUBURBS;\n</P>\n<P>\nDROUGHT: FOUR DRY YEARS HAVE PARCHED NATIVE VEGETATION, FORCING BOBCATS, BEARS,\nMOUNTAIN LIONS, DEER AND COYOTES TO FORAGE CLOSER TO INHABITED AREAS.\n</P>\n</HEADLINE>\n<TEXT>\n<P>\nHungry bobcats, bears and mountain lions -- unable to find food in Ventura\nCounty\'s drought-parched forests -- are being pushed out of their natural\nhabitats to scavenge in rural communities, game officials said Wednesday.\n</P>\n<P>\nTwo weeks ago, a black bear ripped the door off a trailer home in Rose Valley\njust north of Ojai. And within the past month, there have been several reports\nof mountain lions eating livestock near Los Padres National Forest. Several\nbobcats have been reported near houses in the Ojai Valley.\n</P>\n<P>\nAuthorities say that over the past two years they have received twice the\ncomplaints -- about 20 a month -- of wild ani

The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [5]:
from pyserini.index import IndexReader
from IPython.core.display import display, HTML

reader = IndexReader.from_prebuilt_index('robust04')

doc = reader.doc('LA092790-0015').raw()
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [ ]:
analyzed = reader.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

The index also stores the raw document vector, which we can obtain as a Python dictionary of analyzed terms to counts (i.e., term frequency).
For brevity, we only look at terms that appear more than once:

In [ ]:
doc_vector = reader.get_document_vector('LA092790-0015')
{ k: v for (k, v) in doc_vector.items() if v >1 }

{'advis': 2,
 'anim': 9,
 'area': 4,
 'attack': 3,
 'author': 3,
 'bear': 4,
 'been': 11,
 'black': 2,
 'bobcat': 4,
 'california': 2,
 'cat': 3,
 'counti': 4,
 'coyot': 10,
 'debusscher': 3,
 'deer': 3,
 'depart': 2,
 'drought': 4,
 'dry': 2,
 'elsewher': 2,
 'especi': 2,
 'famili': 2,
 'few': 2,
 'food': 3,
 'forest': 2,
 'game': 2,
 'ha': 3,
 'habitat': 2,
 'have': 16,
 'he': 2,
 'hi': 2,
 'hous': 3,
 'hungri': 3,
 'i': 2,
 'jenk': 4,
 'just': 3,
 'leav': 2,
 'lion': 3,
 'live': 2,
 'lot': 2,
 'month': 4,
 'more': 3,
 'mountain': 3,
 'natur': 2,
 'near': 3,
 'off': 2,
 'offici': 5,
 'ojai': 3,
 'on': 2,
 'out': 4,
 'parch': 2,
 'park': 2,
 'past': 2,
 'peopl': 2,
 'rees': 3,
 'report': 4,
 'resid': 2,
 'rural': 3,
 'sai': 2,
 'said': 19,
 'seen': 3,
 'septemb': 2,
 'sever': 3,
 'she': 3,
 "they'r": 5,
 'two': 3,
 'vallei': 4,
 'ventura': 4,
 'we': 2,
 'who': 2,
 'wild': 3,
 'wildlif': 2,
 'would': 2,
 'yard': 2,
 'year': 3}

## Evaluating Ranked Results

We can load some standard evaluation sets such as Robust04, which contains 250 queries, or "topics" as the TREC conferences call them.

In [ ]:
from pyserini.search import get_topics
topics = get_topics('robust04')
print(f'{len(topics)} queries total')

250 queries total


The topics are in a dictionary, whose keys are integers uniquely identifying each query. Each topic contains the following fields:

* `title`: TREC's term for the brief query a user might actually type;
* `description`: a longer form of the query in the form of a complete sentence; and
* `narrative`: a description of what the user is looking for and what kinds of results would be relevant or non-relevant.

In [ ]:
topics[301]

{'description': 'Identify organizations that participate in international criminal activity, the activity, and, if possible, collaborating organizations and the countries involved.',
 'narrative': 'A relevant document must as a minimum identify the organization and the type of illegal activity (e.g., Columbian cartel exporting cocaine). Vague references to international drug trade without identification of the organization(s) involved would not be relevant.',
 'title': 'International Organized Crime'}

For the purpose of your experiments, we'll divide them into a development and test set.

In [ ]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

Now, we'll fetch the relevance judgments for the Robust04 queries, which TREC calls "qrels".

In [ ]:
from urllib.request import urlopen

qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
qrels = []
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  qrels.append([int(qid), 0, docid.decode('UTF-8'), int(score)])
#qrels = [line.strip().split() for line in urlopen(qfile)]

Each record in the qrel contains four fields:

1. the numeric identifier of the query;
2. the round of relevance feedback, which is here always 0;
3. the identifier of a documennt that has been judged; and
4. the relevance score of that document.

In Robust04, all relevance judgments are binary, i.e., 1 or 0. Note that not all non-relevant documents are recorded. The qrel file only contains those documents the annotators actually looked at; the vast majority of documents in the collection have not been judged. In IR evaluation, we assume that unannotated documents are non-relevant.

In [ ]:
qrels[0:10]

[[301, 0, 'FBIS3-10082', 1],
 [301, 0, 'FBIS3-10169', 0],
 [301, 0, 'FBIS3-10243', 1],
 [301, 0, 'FBIS3-10319', 0],
 [301, 0, 'FBIS3-10397', 1],
 [301, 0, 'FBIS3-10491', 1],
 [301, 0, 'FBIS3-10555', 0],
 [301, 0, 'FBIS3-10622', 1],
 [301, 0, 'FBIS3-10634', 0],
 [301, 0, 'FBIS3-10635', 0]]

## Computing Mean Average Precision

The Robust04 collection uses binary relevance judgments and usually has multiple relevant results for each query. It is thus common to use **mean average precision** (MAP) to evaluate retrieval performance on it. Remember from class that MAP adds the precision at the position of each _relevant_ document in a ranked list and then divides by the total number of relevant documents. So that we don't have to scan through the entire collection, we usually evaluate MAP at some maximum rank value, such as 100 or 1000. We simply stop scanning at that maximum rank.

As we saw above, you should pass a query string (the `title` of a topic) and the desired number of results to the `search` method of the `searcher` object.

In [ ]:
hits = searcher.search(dev_topics[355]['title'], 1000)
[(hit.docid, hit.score) for hit in hits[0:10]]

[('FBIS4-20436', 11.349300384521484),
 ('FBIS3-23683', 9.126500129699707),
 ('FBIS3-21238', 8.309300422668457),
 ('FBIS4-44915', 7.935699939727783),
 ('FBIS4-20602', 7.6006999015808105),
 ('FBIS4-47382', 7.529600143432617),
 ('FT943-1589', 7.480100154876709),
 ('LA071789-0059', 7.451399803161621),
 ('FBIS4-22145', 7.215099811553955),
 ('FBIS4-44667', 7.106500148773193)]

For this assignment, evaluate MAP@1000 for the list of `test_topics` we created above. You should process the `qrels` data to find the relevant results for each query.

In [ ]:
## TODO [35 points]: Compute MAP@1000 for test_topics

## Implementing a Query Likelihood Model

The default `SimpleSearcher` in pyserini uses a BM25 model. In this part of the assignment, you'll interact directly with the index to implement a query likelihood retrieval model.  For more details on pyserini's index API, [see the documentation](https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md).

We've already created an `IndexReader` for the Robust04 collection and assigned it to `reader`. Remember that we can pass documents or queries through the index's predefined analysis pipeline, a series of operations like stemming, stopword removal, etc.

In [17]:
reader.analyze('Are there black bear attacks there?')

['black', 'bear', 'attack']

For most retrieval models, including BM25 and query likelihood, we'll want to know the overall statistics for the terms we retrieve from the index. By default, pyserini will do stemming and other "analysis" steps on the terms you give it. But since we're already working with stemmed queries, we'll tell it not to perform this analysis now.

In [12]:
# Just checking that our term is already in its index form.
print(reader.analyze('bear'))

df, cf = reader.get_term_counts('bear', analyzer=None)
print(f'The document frequency is {df} and the collection frequency is {cf}.')

['bear']
The document frequency is 16429 and the collection frequency is 23545.


You might also want to know summary statistics about the collection.

In [25]:
reader.stats()

{'total_terms': 174540872,
 'documents': 528030,
 'non_empty_documents': 528030,
 'unique_terms': 923436}

Now let's traverse the postings for a term.

In [32]:
posting_list = reader.get_postings_list('bear', analyzer=None)
print(f'There are {len(posting_list)} postings.')

posting = posting_list[100]
print(f'Each posting contains a document ID ({posting.docid}), term frequency ({posting.tf}), and position list ({posting.positions}).')

print(f'searcher has a method for turning internal integer IDs like {posting.docid} into external IDs like {searcher.doc(posting.docid).docid()}.')

There are 16429 postings.
Each posting contains a document ID (3330), term frequency (2), and position list ([35, 70]).
searcher has a method for turning internal integer IDs like 3330 into external IDs like LA101389-0041.


If, instead of mapping terms to documents, you want to use a document name to get term information, see above for the `get_document_vector` method.

Now you should have all the information you need to implement a query likelihood retrieval model. Use Dirichlet smoothing with $\mu = 550$.

In [ ]:
## TODO [50 points]: Implement a query likelihood model with Dirichlet smoothing.
## For a given query, index reader, and constant k, return the k documents with the highest query likelihoods.
## Think about what data s would let you keep the top k.

Now you can reuse some of your evaluation code for mean average precision from above.

In [ ]:
## TODO [15 points]: Compute the MAP@1000 for the test set you used above to evaluate BM25.